<a href="https://colab.research.google.com/github/DHasanC/MLHW1/blob/master/MachineLearning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from keras.datasets import cifar10
from keras import Sequential
from keras.utils import np_utils
from keras.optimizers import Adagrad, Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Activation, Flatten, BatchNormalization
from keras import regularizers

Using TensorFlow backend.


In [2]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

## Creating validation set from training dataset

indices = np.random.permutation(len(train_images))

val_indices = indices[0:10000]
train_indices = indices[10000:]

val_images, val_labels = train_images[val_indices], train_labels[val_indices]

Train_images, Train_labels = train_images[train_indices], train_labels[train_indices]


## Normalizing the Train, test and validation sets

X_train = (Train_images/255).astype('float32')
y_train = np_utils.to_categorical(Train_labels, 10)

X_val = (val_images/255).astype('float32')
y_val = np_utils.to_categorical(val_labels, 10)

X_test = (test_images/255).astype('float32')
y_test = np_utils.to_categorical(test_labels, 10)

170500096/170498071 [==============================] - 16s 0us/step


In [3]:
model1 = Sequential()
model1.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))


model1.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model1.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
#model1.add(Dropout(0.5))
 
model1.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model1.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))

model1.add(Flatten())
#model.add(BatchNormalization())
#model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01))
model1.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model1.add(Dense(64, activation= 'relu'))
model1.add(Dense(10, activation='softmax'))

model1.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 48)        13872     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        27712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv

In [0]:
# Initiate Adagrad/RMSprop optimizer
model1.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

## Train the model and get the training and validation loss
history = model1.fit(X_train, y_train, batch_size= 40, epochs= 120,
                    verbose=1, validation_data=(X_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/120
28480/40000 [====================>.........] - ETA: 1:52 - loss: 14.5449 - acc: 0.1000

In [0]:
## Score trained model.
scores = model1.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [0]:
model2 = Sequential()
model2.add(Conv2D(32, (3,3), strides = (1,1), padding='same', activation ='relu', input_shape =(32,32,3),kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))


model2.add(Conv2D(48, (3,3), strides = (1,1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3,3), strides= (1,1),padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(MaxPooling2D(pool_size=(2,2), padding= 'same'))
model1.add(Dropout(0.5))
 
model2.add(Conv2D(128, (3,3), strides= (2,2), padding='same', activation = 'relu',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model2.add(Dropout(0.25))

model2.add(Conv2D(256, (3,3), strides= (2,2), padding='same', activation = 'relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.0001)))
model2.add(Dropout(0.1))

model2.add(Flatten())
model2.add(Dense(128, activation='relu',  kernel_regularizer= regularizers.l2(0.0001)))
model2.add(Dense(64, activation= 'relu'))
model2.add(Dense(10, activation='softmax'))

model2.summary()